# 0. Default start

In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

In [3]:
np.random.seed(42)

In [4]:
df = pd.read_csv('train_dataset_Самолет.csv')

C:\Users\Artem\AppData\Local\Temp\ipykernel_17832\1363469232.py:1: DtypeWarning: Columns (51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,131,132,133,135,136,139,140,141,142,143,144,145,146,147,148,149,151,152,155,156,157,158,159,160,161,163,164,165,167,168,169,171,172,173,174,175,176,177,178,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,223,224,227,239,240,241,242,243,244,245,246,459,460,461,467,468,469,515,516,523,524,531,532,539,540,547,548,555,556,563,564,565,567,571,572,573,575,579,580,581,582,583,587,588,595,596,603,604,611,612,613,615,795,796,797,799,821,822,825,826,1031,1032,1033,1034,1035,1036,1037,1039,1040,1041,1042,1043,1044,1045,1047,1048,1049,1050,1051,1052,1055,1056,1057,1058,1059,1060,1063,1064,1065,1067,1068,1069,1072,1073,1074,1075,1076,1077,1078,1172,1173,1174,1176,1177,1178,1179,1180,1181,1182,1275,1276,1277,1278,1279,1280,1281,1282,14

# 1. Data analyze

In [5]:
def count_missing_values(df):
    """
    Функция для подсчета пропусков в указанном столбце DataFrame. 
    Параметры:
    df (pd.DataFrame): Исходный DataFrame. 
    Возвращает:
    float: Процент пропусков в столбце."""
    total_rows = df.shape[0]  # Общее количество строк в DataFrame
    missing_values = df.isnull().sum()  # Подсчет пропусков в столбце

    # Вычисление процента пропусков
    missing_percentage = (missing_values / total_rows) * 100

    return missing_percentage

In [6]:
df = df.replace({'nan': None})

In [7]:
df.shape

(14456, 2666)

In [8]:
df = df.T.drop_duplicates().T
df.shape

(14456, 2213)

In [9]:
miss_pos_col = count_missing_values(df[df['target'] == 1])
threshold = 75 # percent value in col for drop
to_drop = miss_pos_col[miss_pos_col >= threshold].index.tolist()
len(to_drop)

1876

In [10]:
df = df.drop(to_drop, axis=1)

In [11]:
df.shape

(14456, 337)

In [12]:
def intersection_col(df, list_cols):
    if isinstance(list_cols, list):
        return list(set(df.columns.tolist()) & set(list_cols))
    else:
        return list(set(df.columns.tolist()) & set(list_cols.columns.tolist()))

In [13]:
cat_all_columns = ['col520', 'col528', 'col536',
 'col544', 'col552', 'col592', 'col600', 'col608', 'col1454']
to_drop_cat = ['col552', 'col1454']

In [14]:
cat_cols = intersection_col(df, cat_all_columns)
to_drop = list(set(to_drop_cat) & set(cat_cols))
cat_cols = list(set(cat_cols) - set(to_drop))
len(cat_cols)

0

In [15]:
df = df.drop(to_drop, axis=1)

In [16]:
df.shape

(14456, 336)

In [17]:
cols = df.columns.tolist()
cols.remove('client_id'), cols.remove('target'), cols.remove('report_date')
df[cols] = df[cols].astype(np.float64)

category_cols = []

for col in df.columns.tolist():
    if col == 'report_date':
        continue
    if df[col].max() <= 40:
        category_cols.append(col)
category_cols.remove('col2663'), category_cols.remove('target')

(None, None)

In [18]:
# df[category_cols] = df[category_cols].astype(str)

# 2. Model builder

In [19]:
from sklearn.metrics import classification_report
from eli5.sklearn import PermutationImportance
from lightgbm import LGBMClassifier
from category_encoders import CatBoostEncoder

In [20]:
from sklearn.metrics import (
    roc_auc_score,
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    confusion_matrix,
    average_precision_score,
    log_loss,
    )
def evaluate_metrics(y_true, y_pred):
    roc_auc = roc_auc_score(y_true, y_pred) 
    accuracy = accuracy_score(y_true, (y_pred > 0.5).astype(int)) 
    precision = precision_score(y_true, (y_pred > 0.5).astype(int)) 
    recall = recall_score(y_true, (y_pred > 0.5).astype(int))
    f1 = f1_score(y_true, (y_pred > 0.5).astype(int)) 
    confusion = confusion_matrix(y_true, (y_pred > 0.5).astype(int)) ок
    pr_auc = average_precision_score(y_true, y_pred) 
    logloss = log_loss(y_true, y_pred)

    print(f"ROC AUC: {roc_auc}", end='; ')
    print(f"Accuracy: {accuracy}", end='; ')
    print(f"Precision: {precision}", end='; ')
    print(f"Recall: {recall}", end='; ')
    print(f"F1 Score: {f1}", end='; ')
    print(f"PR AUC: {pr_auc}", end='; ')
    print(f"Log Loss: {logloss}")

In [144]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, ClassifierMixin

from eli5.sklearn import PermutationImportance
from imblearn.over_sampling import SMOTE
import optuna
from optuna.integration import LightGBMPruningCallback
import lightgbm as lgb

    
class FeatureSelector(BaseEstimator, ClassifierMixin):
    def __init__(self, features):
        """
        Класс для отбора заданных признаков.
        
        Параметры:
        features (list): Список заданных признаков.
        """
        self.features = features
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        """
        Метод для отбора заданных признаков из датасета.
        
        Параметры:
        df (pd.DataFrame): Входной датасет.
        
        Возвращает:
        pd.DataFrame: Датасет, содержащий только отобранные признаки.
        
        Исключения:
        ValueError: Генерируется, если какие-то из заданных признаков отсутствуют во входном датасете.
        """
        # Проверяем наличие всех заданных признаков во входном датасете
        missing_features = set(self.features) - set(df.columns)
        if missing_features:
            raise ValueError(f"Признаки {missing_features} отсутствуют во входном датасете.")
        
        # Отбираем заданные признаки из датасета
        selected_df = X[self.features]
        
        return selected_df

def objective(trial, trf, df_train, df_val, target_col='target'):
    params = {
        'objective': 'binary',
        'metric': 'auc',
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.1, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.1, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 100),
        'min_sum_hessian_in_leaf ': trial.suggest_float('min_sum_hessian_in_leaf', 0.0, 10.0),
        'is_unbalance': trial.suggest_categorical('is_unbalance', [True]),
        'random_state': trial.suggest_categorical('random_state', [42])
    }
    pruning_callback = LightGBMPruningCallback(trial, "auc")
    trf = trf.fit(df_train, df_train[target_col])

    model = lgb.train(params, lgb.Dataset(trf.transform(df_train), label=df_train[target_col]), num_boost_round=1000,
                    valid_sets=[lgb.Dataset(trf.transform(df_val), label=df_val[target_col])], 
                    early_stopping_rounds=100, verbose_eval=False, callbacks=[pruning_callback])
   
    y_pred = model.predict(trf.transform(df_val))

    evaluate_metrics(df_val[target_col], y_pred)
    roc_auc = roc_auc_score(df_val[target_col], y_pred) 

    return roc_auc
    
class ModelBuilder(object):
    def __init__(self, cat_cols, features_name, ml_cfg, target_col='target', params=None):
        
        self.features = features_name
        self.cat_cols = cat_cols
        self.target_col = target_col
        self.cfg = ml_cfg
        selector = FeatureSelector(features=features_name)
        enc = CatBoostEncoder(cols=cat_cols)
        lgbm = LGBMClassifier()
        self.model = Pipeline([('sel', selector), ('enc', enc), ('lgbm', lgbm)])      
        self.model.set_params(**params) 
        
        
    def train(self, df_train):
        model = self.model.fit(df_train, df_train[self.target_col])
        return model
  
    def get_transformer_data(self):
        return Pipeline(self.model.steps[:-1])
        
    def select_features(self, df_train, df_val):

        self.train(df_train)
        transformer = self.get_transformer_data()
        model = self.model.steps[-1][-1]
        transformed_val_data = transformer.transform(df_val)

        print('start feature select...')
        perm = PermutationImportance(model, random_state=42) \
                                    .fit(transformed_val_data, df_val[self.target_col])
        print('end feature select...')

        feature_importances = list(zip(transformed_val_data.columns, perm.feature_importances_))
        feature_importances.sort(key=lambda x: x[1], reverse=True)
        selectd_featurs = [name_col for name_col, value in feature_importances if value > 0]
        print(f'selected {len(selectd_featurs)} out of {len(df_train.columns)}')
        
        cat_cols = list(set(self.cat_cols) & set(selectd_featurs))
        self.model.set_params(**{'sel__features': selectd_featurs,
                                'enc__cols': cat_cols})
    
    def params_tuning(self, df_train, df_val):
        print('start tuning...')
        study = optuna.create_study(direction='maximize')
        transformer = self.get_transformer_data()
        study.optimize(lambda trial: objective(trial, transformer, df_train, df_val),
                       n_jobs=1, n_trials=self.cfg['n_trails'])
        print('end tuning...')

        best_params = study.best_params
        best_score = study.best_value
        best_params = {f'lgbm__{k}':v for k,v in best_params.items()}
        self.model.set_params(**best_params)
        print(f'best params: {best_params}')
        print(f'best score: {best_score}')
    
    # def sampling_dataset(self, x, y):
    #     print('start sampling with SMOTE ...')
    #     trf = self.get_transformer_data().fit(x, y)
    #     sampler = SMOTE(sampling_strategy='auto', random_state=42)
    #     X_train_resampled, y_train_resampled = sampler.fit_resample(trf.transform(x.fillna(-1)), y)
    #     return X_train_resampled, y_train_resampled
    
    def build(self, df):
        print('start build model...')
        df_train, df_val = train_test_split(df, random_state=42, test_size=self.cfg['test_size'],
                                            shuffle=True, stratify=df[self.target_col])
        
        self.select_features(df_train, df_val)
        
        self.params_tuning(df_train, df_val)

        self.model.fit(df_train, df_train[self.target_col])
        
        y_pred = self.model.predict(df_val)
        metrics = roc_auc_score(df_val[self.target_col], y_pred)
        print(f'roc_auc on validate data = {metrics}')
        
        return self.model

# 3. Create and get trainable model

In [149]:
exclude_cols = ['target', 'client_id', 'report_date']
features = df.columns.tolist()
features = list(set(features) - set(exclude_cols))
category_cols = list(set(features) & set(category_cols))
default_params = {'lgbm__n_estimators': 100, 'lgbm__max_depth': 10, 
                  'lgbm__random_state': 42, 'lgbm__n_jobs': -1}
ml_cfg =  {'n_trails': 100, 'test_size': 0.25}

In [150]:
df['target'] = df['target'].astype(int)

In [151]:
builder = ModelBuilder(cat_cols = category_cols, features_name=features, ml_cfg=ml_cfg, params=default_params)

In [152]:
model = builder.build(df)

start build model...
start feature select...


[I 2023-09-09 21:10:04,357] A new study created in memory with name: no-name-eeb6bf39-0271-4eca-9aa2-715848b80a70


end feature select...
selected 88 out of 336
start tuning...


[I 2023-09-09 21:10:05,129] Trial 0 finished with value: 0.8764203169783665 and parameters: {'num_leaves': 98, 'learning_rate': 0.03708942446287805, 'feature_fraction': 0.17052261372406863, 'bagging_fraction': 0.35884062766941294, 'bagging_freq': 8, 'max_depth': 3, 'min_child_samples': 33, 'min_sum_hessian_in_leaf': 2.0667689890434837, 'is_unbalance': True, 'random_state': 42}. Best is trial 0 with value: 0.8764203169783665.


ROC AUC: 0.8764203169783665; Accuracy: 0.9222468179302712; Precision: 0.23939393939393938; Recall: 0.7247706422018348; F1 Score: 0.35990888382687924; PR AUC: 0.34264731153556455; Log Loss: 0.20487345956900088


...

ROC AUC: 0.9019709720059156; Accuracy: 0.9698395130049806; Precision: 0.0; Recall: 0.0; F1 Score: 0.0; PR AUC: 0.3942588098725532; Log Loss: 0.11255295617047217


[I 2023-09-09 21:11:22,115] Trial 97 pruned. Trial was pruned at iteration 80.
C:\Users\Artem\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2023-09-09 21:11:23,314] Trial 98 finished with value: 0.8971901215825361 and parameters: {'num_leaves': 124, 'learning_rate': 0.023233657231561452, 'feature_fraction': 0.4339722885989097, 'bagging_fraction': 0.8626743522874187, 'bagging_freq': 5, 'max_depth': 10, 'min_child_samples': 9, 'min_sum_hessian_in_leaf': 3.7057090780398503, 'is_unbalance': True, 'random_state': 42}. Best is trial 81 with value: 0.9099032836445969.


ROC AUC: 0.8971901215825361; Accuracy: 0.9698395130049806; Precision: 0.0; Recall: 0.0; F1 Score: 0.0; PR AUC: 0.395287612481583; Log Loss: 0.10888544577016179


[I 2023-09-09 21:11:23,818] Trial 99 pruned. Trial was pruned at iteration 0.


end tuning...
best params: {'lgbm__num_leaves': 176, 'lgbm__learning_rate': 0.037676630634207287, 'lgbm__feature_fraction': 0.4267394208152833, 'lgbm__bagging_fraction': 0.9761636622221321, 'lgbm__bagging_freq': 4, 'lgbm__max_depth': 12, 'lgbm__min_child_samples': 15, 'lgbm__min_sum_hessian_in_leaf': 3.667235736293018, 'lgbm__is_unbalance': True, 'lgbm__random_state': 42}
best score: 0.9099032836445969
roc_auc on validate data = 0.719492468164745


# 4. Submission file

In [ ]:
test = pd.read_csv(...)

In [ ]:
test['score'] = model.predict_proba(test)

In [ ]:
sample_submission = test.loc[:, ['report_date', 'client_id', 'score']]

In [ ]:
sample_submission.to_csv('result_novichki.csv')